In [58]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import requests
import nltk
from nltk import tokenize
from fuzzywuzzy import fuzz
from bs4 import BeautifulSoup
from sklearn import preprocessing
from sklearn import model_selection



In [81]:
URL = r'https://batdongsan.vn/ban-nha/'

# parse  string list[dict{str: str}]
dict_html_parsing = [{'body':''},\
                    {'div': 'landtech-container'},\
                    {'div':  "uk-container uk-container-center"},\
                    {'div': "block-custom realestate-post-detail-custom"},\
                    {'div': "uk-grid uk-grid-medium"},\
                    {'div': "uk-width-medium-3-5"},\
                    {'div': "project-global-object-block-003 information-custom"},\
                    {'div': "wrapper"},\
                    {'div': "uk-panel"}]

dict_html_house_attribs = dict_html_parsing
dict_html_house_title = dict_html_parsing
dict_html_house_price = dict_html_parsing

dict_html_house_title.extend([{'h1': "uk-panel-title"},\
                                                {'span':''}])

dict_html_house_price.extend([{'div': "item"},\
                                                {'div': "body"},\
                                                {'div': "meta"},\
                                                {'strong': "price"}])

dict_html_house_attribs.extend([{'div': "item"},\
                                                {'div': "body"},\
                                                {'div': "param"},\
                                                {'ul', "uk-list"},\
                                                {'li': ''}])

header_features = ['Price', 'Title', 'Address', 'Area', 'Room', 'WC' , 'Des']

for item in dict_html_house_title:
    print(list(item.keys())[0])


body
div
div
div
div
div
div
div
div
h1
span
div
div
div
strong
div
div
div


AttributeError: 'set' object has no attribute 'keys'

# Data Collection 


In [42]:
class Address:
    def __init__(self, city, district, ward, street):
        self._city = city
        self._district = district
        self._ward = ward
        self._street = street
    
    def display(self):
        print(self._street, self._ward, self._district, self._city)

In [74]:
class House:
    def __init__(self, title,  price ,address, area, room, wc, description):
        self._title = title
        self._price = price
        self._address = address
        self._area= area 
        self._room = room 
        self._wc = wc
        self._description = description
    
    def scrappingDataWeb(self, url):
        page = requests.get(url)
        
        # title parsing
        soup_title = BeautifulSoup(page.content, "html.parser")
        print(soup_title)
        for item in dict_html_house_title:
            if item.get(list(item.keys())[0]) != '':
                temp = soup_title.find(list(item.keys())[0], class_ = item.get(list(item.keys())[0]))
                soup_title = temp
            else:
                temp = soup_title.find(list(item.keys())[0])
                soup_title = temp
        self._title = soup_title.text
        
        # price parsing
        try:
            soup_price = BeautifulSoup(page.content, "html.parser")
            for item in dict_html_house_price:
                if item.get(list(item.keys())[0]) != '':
                    soup_price = soup_price.find(list(item.keys())[0], class_ = item.get(list(item.keys())[0]))
                else:
                    soup_price = soup_price.find(list(item.keys())[0])
            self._price = soup_price.text
        except:
            print("error parsing price: ", url)
        
        ## parsing another attributes
        try:
            soup_attrib = BeautifulSoup(page.content, "html.parser")
            for item in dict_html_house_attribs:
                if item.get(list(item.keys())[0]) != '':
                    soup_attrib = soup_attrib.find(list(item.keys())[0], class_ = item.get(list(item.keys())[0]))
                else:
                    soup_attrib = soup_attrib.find(list(item.keys())[0])
            attrib_root = soup_attrib.find_all("i")
            for root in attrib_root:
                item = root.find("strong")
                if item.innerHTML =="Diện tích:":
                    self._area = item.text
                elif item.innerHTML =="Phòng ngủ:":
                    self._room == item.text
                elif item.innerHTML =="Địa chỉ:":
                    self._address = item.text
        except:
            print("error parsing attribs: ", url)
            
        ## chỉnh sửa phần này thành nhiều thành phần attib
        
    
    ## attributes matching for finding address base on The compare with voacabulary embedded database with Levhein distance
    def tranferDescription(self, embedded_database): 
        temp_address = Address('','','','')
        for city in embedded_database:
            for district in city._container:
                ## tokenize description scrapping from web to list vocab
                for token in tokenize(self._description):
                    # compare each vocab with each item address from db
                    score = fuzz(token, district)
                    if score >= 0.7:
                        Address._district = district
        return
    
    ##
    def tranferAddress(self):
        return
    def display(self):
        print(self._title,\
              self._price,\
              self._address,\
              self._area,\
              self._room,\
              self._wc,\
              self._description)

In [75]:
temp = House("","","","","","","")
temp.scrappingDataWeb("https://batdongsan.vn/ban-nha-pho-cu-loc-thanh-xuan-56m-4-tang-gia-545-ty-r268420")

<!DOCTYPE html>

<html id="post-detail" lang="vi">
<head>
<title>Bán nhà phố Cự Lộc, Thanh Xuân 56m, 4 tầng. Giá: 5.45 tỷ</title>
<base href="https://batdongsan.vn/"/>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="index,follow,all" name="robots"/>
<meta content="" name="keywords"/>
<meta content='"Bán nhà phố Cự Lộc, Thanh Xuân 56m, 4 tầng. Giá: 5.45 tỷBán nhà phố Cự Lộc, nhà 4 tầng đầy đủ tiện nghi, về ở ngayNhà vị trí đẹp, nằm trên mặt ngõ đẹp nhất phố Cự Lộc, kinh doanh tấp...' name="description"/>
<meta content="batdongsan.vn" name="generator"/>
<meta content="vi_VN" property="og:locale"/>
<meta content="Bán nhà phố Cự Lộc, Thanh Xuân 56m, 4 tầng. Giá: 5.45 tỷ" property="og:title"/>
<meta content='"Bán nhà phố Cự Lộc, Thanh Xuân 56m, 4 tầng. Giá: 5.45 tỷBán nhà phố Cự Lộc, nhà 4 tầng đầy đủ tiện nghi, về ở ngayNhà vị trí đẹp, nằm trên mặt ngõ đẹp nhất phố Cự Lộc, ki

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
class ContainerHouse:
    def __init__(self, _list = []):
        self._size = len(self._list)
    